### Import necessary Libraries

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... failed



CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/noarch/repodata.json.bz2>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

ConnectionError(MaxRetryError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Max retries exceeded with url: /pkgs/main/noarch/repodata.json.bz2 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000000DB147204E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))",),)




Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'W43U5VPP2BW1ZGWTMMRKXLIXSFNTMJACKBYA525IO4QMWOAZ' # your Foursquare ID
CLIENT_SECRET = 'URJ44EE2ZIFUV3BZLELJRMMMHY2VUTWCP5J43FPQAWM3NGTF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 10000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W43U5VPP2BW1ZGWTMMRKXLIXSFNTMJACKBYA525IO4QMWOAZ
CLIENT_SECRET:URJ44EE2ZIFUV3BZLELJRMMMHY2VUTWCP5J43FPQAWM3NGTF


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

C:\Users\jainav01\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


40.7149555 -74.0153365


##  Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [4]:
latitude = 40.715337
longitude = -74.008848

#### Define URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=W43U5VPP2BW1ZGWTMMRKXLIXSFNTMJACKBYA525IO4QMWOAZ&client_secret=URJ44EE2ZIFUV3BZLELJRMMMHY2VUTWCP5J43FPQAWM3NGTF&ll=40.715337,-74.008848&v=20180605&radius=10000&limit=100'

#### Send GET request and examine results

In [6]:
import requests

In [7]:
results = requests.get(url).json()
'There are {} around Conrad Hotel.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Conrad Hotel.'

#### Get relevant part of JSON

In [8]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-537b5258498e262a54b47216-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'primary': True,
    'shortName': 'Bakery'}],
  'id': '537b5258498e262a54b47216',
  'location': {'address': '220 Church St',
   'cc': 'US',
   'city': 'New York',
   'country': 'United States',
   'crossStreet': 'btwn Worth & Thomas St',
   'distance': 270,
   'formattedAddress': ['220 Church St (btwn Worth & Thomas St)',
    'New York, NY 10013',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71707213063327,
     'lng': -74.00660521707641}],
   'lat': 40.71707213063327,
   'lng': -74.00660521707641,
   'postalCode': '10013',
   'state': 'NY'},
  'n

#### Process JSON and convert it to a clean dataframe

In [9]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']  
    except:
        categories_list = row['venue.categories'] 
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Arcade Bakery,Bakery,220 Church St,US,New York,United States,btwn Worth & Thomas St,270,"[220 Church St (btwn Worth & Thomas St), New Y...","[{'label': 'display', 'lat': 40.71707213063327...",40.717072,-74.006605,NaN,10013,NY,537b5258498e262a54b47216
1,Washington Market Park,Playground,Greenwich St,US,New York,United States,at Reade St,268,"[Greenwich St (at Reade St), New York, NY, Uni...","[{'label': 'display', 'lat': 40.71704598853704...",40.717046,-74.011095,NaN,NaN,NY,4a9bcc2ff964a5203b3520e3
2,Aire Ancient Baths,Spa,88 Franklin St,US,New York,United States,at Church St,453,"[88 Franklin St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71814141164123...",40.718141,-74.004941,NaN,10013,NY,4fbbd9ede4b0756c0d4c2364
3,One World Trade Center,Building,1 World Trade Ctr,US,New York,United States,btwn Fulton & West St,440,"[1 World Trade Ctr (btwn Fulton & West St), Ne...","[{'label': 'display', 'lat': 40.71306941116871...",40.713069,-74.013133,NaN,10007,NY,4abe7f9ef964a520068e20e3
4,The Bar Room at Temple Court,Hotel Bar,123 Nassau St,US,New York,United States,NaN,466,"[123 Nassau St, New York, NY 10038, United Sta...","[{'label': 'display', 'lat': 40.7114477287544,...",40.711448,-74.006802,Financial District,10038,NY,57f0689d498e7d49d9189369
5,THE BEEKMAN - A Thompson Hotel,Hotel,123 Nassau St,US,New York,United States,at Beekman St,498,"[123 Nassau St (at Beekman St), New York, NY 1...","[{'label': 'display', 'lat': 40.71113796121251...",40.711138,-74.006795,NaN,10038,NY,56d8c0f8498edb854f926e6a
6,9/11 Memorial North Pool,Fountain,West St,US,New York,United States,at Fulton St,515,"[West St (at Fulton St), New York, NY, United ...","[{'label': 'display', 'lat': 40.71207726282092...",40.712077,-74.013187,NaN,NaN,NY,58801864cc5b6a14dedce689
7,Locanda Verde,Italian Restaurant,377 Greenwich St,US,New York,United States,at N Moore St,534,"[377 Greenwich St (at N Moore St), New York, N...","[{'label': 'display', 'lat': 40.72004087043341...",40.720041,-74.010116,NaN,10013,NY,4a149749f964a52055781fe3
8,Pier 25 - Hudson River Park,Pier,West Side Highway,US,New York,United States,at N Moore St,641,"[West Side Highway (at N Moore St), New York, ...","[{'label': 'display', 'lat': 40.72019260471762...",40.720193,-74.012950,NaN,10013,NY,4ba0e4e3f964a520498537e3
9,Nelson A. Rockefeller Park,Park,North end of Battery Park City & West of River...,US,New York,United States,at Warren St,692,[North end of Battery Park City & West of Rive...,"[{'label': 'display', 'lat': 40.71709477942235...",40.717095,-74.016716,NaN,10280,NY,4b929790f964a5209a0734e3


#### Let's visualize these items on the map around our location

In [10]:
dataframe_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
name                100 non-null object
categories          100 non-null object
address             99 non-null object
cc                  100 non-null object
city                100 non-null object
country             100 non-null object
crossStreet         91 non-null object
distance            100 non-null int64
formattedAddress    100 non-null object
labeledLatLngs      90 non-null object
lat                 100 non-null float64
lng                 100 non-null float64
neighborhood        12 non-null object
postalCode          98 non-null object
state               100 non-null object
id                  100 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 12.6+ KB


In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

##   Now as a final step lets get into clustering into 4 key areas

Analyze Each Address

In [14]:
# one hot encoding
ny_onehot = pd.get_dummies(dataframe_filtered[['categories']], prefix="", prefix_sep="")

# add address column back to dataframe
ny_onehot['address'] = dataframe_filtered['address'] 

# move address column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head(10)

,address,American Restaurant,Art Gallery,Art Museum,Bakery,Beach,Beer Store,Bookstore,Bridge,Building,...,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Track,Wine Bar,Wine Shop,Yoga Studio
0,220 Church St,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Greenwich St,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,88 Franklin St,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1 World Trade Ctr,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,123 Nassau St,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,123 Nassau St,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,West St,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,377 Greenwich St,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,West Side Highway,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,North end of Battery Park City & West of River...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
ny_onehot.shape

(100, 62)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
ny_grouped = ny_onehot.groupby('address').mean().reset_index()
ny_grouped

,address,American Restaurant,Art Gallery,Art Museum,Bakery,Beach,Beer Store,Bookstore,Bridge,Building,...,Steakhouse,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Track,Wine Bar,Wine Shop,Yoga Studio
0,1 Union Sq W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1 World Trade Ctr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10 E 8th St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101 Saint Marks Pl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,103 East 19th Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,117 Macdougal St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1170 Broadway,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,119 Sullivan St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,123 Nassau St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,126 E 13th St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
ny_grouped.shape

(95, 62)

# Cluster areas into 4
#### Run *k*-means to cluster the address into 4 clusters.

In [21]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

ny_grouped_clustering = ny_grouped.drop('address', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well

In [23]:
ny_merged = ny_grouped

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

ny_merged.tail() # check the last columns!

,address,American Restaurant,Art Gallery,Art Museum,Bakery,Beach,Beer Store,Bookstore,Bridge,Building,...,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Track,Wine Bar,Wine Shop,Yoga Studio,Cluster Labels
90,Washington Square Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
91,West Side Highway,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
92,West St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
93,btwn Broadway & Union Sq E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
94,btwn Gansevoort & W 34th St,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


Put cluster label in actual dataframe

In [39]:
df_final = dataframe_filtered.merge(ny_grouped[['address','Cluster Labels']], how='left', on='address')
df_final['Cluster Labels'] = df_final['Cluster Labels'].fillna(0).astype('int')
df_final.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,Cluster Labels
0,Arcade Bakery,Bakery,220 Church St,US,New York,United States,btwn Worth & Thomas St,270,"[220 Church St (btwn Worth & Thomas St), New Y...","[{'label': 'display', 'lat': 40.71707213063327...",40.717072,-74.006605,NaN,10013,NY,537b5258498e262a54b47216,0
1,Washington Market Park,Playground,Greenwich St,US,New York,United States,at Reade St,268,"[Greenwich St (at Reade St), New York, NY, Uni...","[{'label': 'display', 'lat': 40.71704598853704...",40.717046,-74.011095,NaN,NaN,NY,4a9bcc2ff964a5203b3520e3,0
2,Aire Ancient Baths,Spa,88 Franklin St,US,New York,United States,at Church St,453,"[88 Franklin St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71814141164123...",40.718141,-74.004941,NaN,10013,NY,4fbbd9ede4b0756c0d4c2364,0
3,One World Trade Center,Building,1 World Trade Ctr,US,New York,United States,btwn Fulton & West St,440,"[1 World Trade Ctr (btwn Fulton & West St), Ne...","[{'label': 'display', 'lat': 40.71306941116871...",40.713069,-74.013133,NaN,10007,NY,4abe7f9ef964a520068e20e3,0
4,The Bar Room at Temple Court,Hotel Bar,123 Nassau St,US,New York,United States,NaN,466,"[123 Nassau St, New York, NY 10038, United Sta...","[{'label': 'display', 'lat': 40.7114477287544,...",40.711448,-74.006802,Financial District,10038,NY,57f0689d498e7d49d9189369,0


Finally, let's visualize the resulting clusters

In [40]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_final['lat'], df_final['lng'], df_final['name'], df_final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters